In [1]:
import torch
import torch.nn
import torch.utils.data
import torch.hub
import torch.optim
import torchvision.transforms
import numpy as np
import os

from tqdm import tqdm

import utils.datasets

device = 'cpu'

In [20]:
class YOLO(torch.nn.Module):

    def __init__(self, bounding_box_shape):
        super(YOLO, self).__init__()
        output_size = bounding_box_shape[0] * bounding_box_shape[1]
        self.convolution = torch.hub.load('pytorch/vision:v0.4.2', 'resnet18', pretrained=True)
        self.bounding_box = torch.nn.Sequential(
            torch.nn.Dropout(),
            torch.nn.Linear(in_features=1000, out_features=4000),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=0.5),
            torch.nn.Linear(in_features=4000, out_features=4000),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=0.5),
            torch.nn.Linear(in_features=4000, out_features=output_size),
        )
        self.bounding_box_shape = bounding_box_shape

    def forward(self, x):
        x = self.convolution(x)
        x = self.bounding_box(x)
        breakpoint()
        x = torch.reshape(x, shape=self.bounding_box_shape)
        return x



In [3]:
PATH_TO_TEST = '../data-science-bowl-2018/stage1_test'
PATH_TO_TRAIN = '../data-science-bowl-2018/stage1_train'

In [4]:
train = utils.datasets.PandasDataset(PATH_TO_TRAIN, augment=True, multiscale=True,normalized_labels=False)


Loading from ../data-science-bowl-2018/stage1_train...
Loading masks...
Loading from ../data-science-bowl-2018/stage1_train...
Loading masks...


Read masks::   0%|          | 2/670 [00:00<00:41, 16.12it/s]

Loaded 670 masks
Loaded 670 masks


Read masks:: 100%|██████████| 670/670 [03:04<00:00,  3.64it/s]


Loading data done!


Loading data done!


In [5]:
# test = utils.datasets.PandasDataset(PATH_TO_TEST, augment=False, multiscale=False)


In [21]:
#because each batch is a single tensor (matrix) 
#and pytorch is saying that all the images 
#in a batch need to be the same size
batch_size = 1#128
learning_rate = 0.0001
epoch = 30

bounding_box_shape = (3, 3, 7)

train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
# test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size)

model = YOLO(bounding_box_shape).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_function = torch.nn.MSELoss()

Using cache found in /Users/kruszylo/.cache/torch/hub/pytorch_vision_v0.4.2
Using cache found in /Users/kruszylo/.cache/torch/hub/pytorch_vision_v0.4.2


In [ ]:
for epoch_number in range(epoch):
    print("Epoch: ", epoch_number)

    model.train()
    for _img_path, x, y in tqdm(train_loader, desc="Training: "):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        output = model(x)
        loss_value = loss_function(output, y)
        loss_value.backward()
        optimizer.step()

    model.eval()

    test_loss = 0

#     with torch.no_grad():
#         for x, y in tqdm(test_loader, desc='Evaluate: '):
#             x, y = x.to(device), y.to(device)
#             output = model(x)
#             test_loss += np.mean(loss_function(output, y).item())

#         print('\nLoss value: {}'.format(test_loss))





Training:   0%|          | 0/29461 [00:00<?, ?it/s]

Epoch:  0
Epoch:  0
> <ipython-input-20-1f5961551314>(23)forward()
-> x = torch.reshape(x, shape=self.bounding_box_shape)
> <ipython-input-20-1f5961551314>(23)forward()
-> x = torch.reshape(x, shape=self.bounding_box_shape)
(Pdb) x.shape
torch.Size([1, 9])
torch.Size([1, 9])
(Pdb) x
tensor([[ 0.1769,  0.1079, -0.1357, -0.0677, -0.3508,  0.3863, -0.0590,  0.2838,
         -0.0392]], grad_fn=<AddmmBackward>)
tensor([[ 0.1769,  0.1079, -0.1357, -0.0677, -0.3508,  0.3863, -0.0590,  0.2838,
         -0.0392]], grad_fn=<AddmmBackward>)
